# Figure out waypoints we want for future field seasons

Priorities
1. Repeat ApRES up the channel
2. New ApRES further up the channel


## Confirm drill hole


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import time
import datetime as dt
import pandas as pd
import geopandas as gpd
import scipy as sp
from scipy import signal
from tqdm.notebook import tqdm

from shapely.geometry import Point, LineString, Polygon, MultiPoint
import fiona
from shapely.ops import nearest_points

In [8]:
bed_gdf = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/PROCESSED_LINES/picked_bed.shp')

In [9]:
def add_distance_to_gdf(gdf):
    """
    Input: geodataframe
    Output: Adds columns 'dx' and 'distance_cum' to the geodataframe
    """
    tmp_dfp = [Point.distance(gdf.geometry.iloc[i]) for i,Point in enumerate(gdf.geometry.iloc[1:])] #note the 1:, equivalent to i+1
    tmp_dfp[:0] = [0]
    gdf['dx'] = pd.Series(tmp_dfp) 
    gdf['distan_cum'] = gdf.dx.cumsum().copy()
    
def add_dice_thickndx(gdf):
    """
    Input: geodataframe
    Output: Adds columns 'dice_thickn' and 'dice_thickndx' to the geodataframe
    """
    
    gdf['dice_thickn'] = gdf.ice_thickn.diff().copy()
        
    gdf['dice_thickndx'] = gdf.dice_thickn.to_numpy()/gdf.dx.to_numpy()

In [10]:
ll = {} #a dictionary of geodataframes of radarlines over KIS2, ie one gdf for each line.

for line in bed_gdf[bed_gdf.camp =='KIS2'].line_name.unique().tolist():
    ll[line] = bed_gdf[(bed_gdf.line_name==line) & (bed_gdf.camp=='KIS2')].copy()
    ll[line].sort_values(by='x',inplace=True)
    ll[line].reset_index(drop=True,inplace=True)
    add_distance_to_gdf(ll[line])
    add_dice_thickndx(ll[line])
    

In [34]:
channel = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/SHARE_SHAPEFILES/interpolated_channel_mid.shp')
channel_ls = LineString(channel.geometry.tolist())

In [11]:
#no line0 
cross_channel_lines = [ 'line7p75','line7p5','line7p25','line7', 'line6','lineold5p5','line5','lineAPREScross','line4','line3','line2','line1',]

In [84]:
channel_intersections = {}
for key,line in ll.items():
    line_ls = LineString(line.geometry.tolist()) #line linestring
    if line_ls.intersects(channel_ls):
        channel_intersections[key] = line_ls.intersection(channel_ls)

In [85]:
channel_intersections['line7p75'] = channel.geometry.iloc[0]

In [86]:
channel_intersections_gdf = gpd.GeoDataFrame({'line_names':list(channel_intersections.keys()),
                                            'intersection_points': list(channel_intersections.values()) },
                                            geometry = 'intersection_points',crs="EPSG:3031")

In [87]:
channel_intersections_gdf.drop([9,11],axis=0).to_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/SHARE_SHAPEFILES/radar_channel_midpoints.shp')

## Comment

Couldnt write the down channel lines as have multiple points and GPD didnt like multipoints

# Notes on waypoints

see /Users/home/whitefar/DATA/FIELD_ANT_19/PRE_FIELD_21/plan_for_ApRES.shp

- Cp16 to Cp25 were single epoch in Dec 2019

- Cp26 to Cp33 extras suggested.

## Triage list

A) First do up channel second epoch. files on plan_for_ApRES.shp:
1. Cp26
2. Cp25
1. Cp24
2. Cp23
1. Cp22
2. Cp21
1. Cp20
2. Cp19
1. Cp18
2. Cp17
1. Cp16


B) Second do first epoch further up channel. Files on plan_for_ApRES.shp
1. Cp27
1. Cp28
1. Cp29_drill
1. Cp30
1. Cp31
1. Cp32
1. Cp33
1. Cp34

C) Third repeat ApRESS cross channel

D) install ApRES at Cp_29_drill
